# Importing The Required Libraries

In [2]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
import difflib
#If spelling mistake happens by user while entering food name then it finds closest match of food given by user from our data
from nltk.stem.porter import PorterStemmer  # This gives rootword to a particular word
from sklearn.feature_extraction.text import TfidfVectorizer # Converts words to Vectors 
from sklearn.metrics.pairwise import cosine_similarity

import warnings

# Disable warning display
warnings.filterwarnings('ignore')

In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Suramya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:

sentiment = np.array([1,1,1,1,1,0,0,0,0,0])

# Importing The Food Dataset

In [6]:
food_df1=pd.read_csv("Recipe Recommendation dataset.csv")
food_df1.head()

,recipe_title,url,vote_count,rating,description,cuisine,course,diet,prep_time,ingredients,instructions,author,tags,category,index
0,Roasted Peppers And Mushroom Tortilla Pizza Re...,https://www.archanaskitchen.com/roasted-pepper...,434,4.958525,is a quicker version pizza to satisfy your cr...,Mexican,Dinner,Vegetarian,15 M,Tortillas|Extra Virgin Olive Oil|Garlic|Mozzar...,To begin making the Roasted Peppers And Mushro...,Divya Shivaraman,Party Food Recipes|Tea Party Recipes|Mushroom ...,Pizza Recipes,1
1,Thakkali Gotsu Recipe,https://www.archanaskitchen.com/tomato-gotsu-r...,3423,4.932223,also known as the is a quick and easy to ma...,South Indian Recipes,Lunch,Vegetarian,10 M,Sesame (Gingelly) Oil|Mustard seeds (Rai/ Kadu...,To begin making Tomato Gotsu Recipe/ Thakkali ...,Archana Doshi,Vegetarian Recipes|Tomato Recipes|South Indian...,Indian Curry Recipes,2
2,Spicy Grilled Pineapple Salsa Recipe,https://www.archanaskitchen.com/spicy-grilled-...,2091,4.945959,Spicy Grilled Pineapple Salsa is a simple reci...,Mexican,Side Dish,Vegetarian,10 M,Extra Virgin Olive Oil|Pineapple|White onion|R...,To begin making the Spicy Grilled Pineapple Sa...,Archana's Kitchen,Party Starter & Appetizer Recipes|Pineapple Re...,Mexican Recipes,3
3,Toor dal Curry,https://www.archanaskitchen.com/dali-thoy-reci...,990,4.888889,The is a quintessential of Konkani dish whic...,Coastal Karnataka,Side Dish,High Protein Vegetarian,5 M,Arhar dal (Split Toor Dal)|Turmeric powder (Ha...,To prepare Karwar Style Dali Thoy Recipe (Toor...,Jyothi Rajesh,Side Dish Recipes|South Indian Recipes|Indian ...,Indian Curry Recipes,4
4,Rajma Kofta In Milk And Poppy Seed Gravy Recipe,https://www.archanaskitchen.com/rajma-kofta-in...,345,4.828986,Koftas are traditional Indian recipes mostly w...,North Indian Recipes,Side Dish,High Protein Vegetarian,20 M,Rajma (Large Kidney Beans)|Cashew nuts|Sultana...,To begin making Rajma Kofta In Milk And Poppy ...,RUBY PATHAK,Side Dish Recipes|Indian Lunch Recipes|Office ...,Kofta Recipes,5


# Data Cleaning

In [7]:
# Removing the Unwanted Symbols
food_df1.replace({"ingredients":{"(":" ",")":" ","|":" ","/":" "} },inplace=True)

In [8]:
food_df1["ingredients"].head()

0    Tortillas|Extra Virgin Olive Oil|Garlic|Mozzar...
1    Sesame (Gingelly) Oil|Mustard seeds (Rai/ Kadu...
2    Extra Virgin Olive Oil|Pineapple|White onion|R...
3    Arhar dal (Split Toor Dal)|Turmeric powder (Ha...
4    Rajma (Large Kidney Beans)|Cashew nuts|Sultana...
Name: ingredients, dtype: object

In [9]:
food_df1["ingredients"]=food_df1["ingredients"].fillna('')

#  Stemming

In [10]:
port_stem=PorterStemmer() #Load PortorStemmer function to port_stem variable

In [11]:
def stemming(text):
  stemmed_text= re.sub('[^a-zA-Z]',' ',text) #Removing Things that are not alphabets like comma,Numbers and all. from text are replaced by space
  stemmed_text= stemmed_text.lower() #Convert all new text to Lower Case bcoz can create problem for ML model
  stemmed_text= stemmed_text.split() #Splitted and coverted to list 
  stemmed_text= [port_stem.stem(word) for word in stemmed_text if not word in stopwords.words('english') ] #Stemming the words that are not in stopwords
  stemmed_text=' '.join(stemmed_text)
  return stemmed_text

In [12]:
#Apply the Above the Created Stemming Function to ingridients Column
new_ingredients1=food_df1["ingredients"].apply(stemming)

In [13]:
new_ingredients1.head()

0    tortilla extra virgin oliv oil garlic mozzarel...
1    sesam gingelli oil mustard seed rai kadugu cur...
2    extra virgin oliv oil pineappl white onion red...
3    arhar dal split toor dal turmer powder haldi s...
4    rajma larg kidney bean cashew nut sultana rais...
Name: ingredients, dtype: object

In [14]:
new_ingredients1.shape

(7963,)

In [15]:
food_df1["Stemmed_ingredients"]=new_ingredients1


In [16]:
food_df1["Stemmed_ingredients"][1]

'sesam gingelli oil mustard seed rai kadugu curri leav garlic pearl onion sambar onion tomato tamarind turmer powder haldi salt jaggeri'

In [17]:
# Get the column names
column_names = food_df1.columns

# Print the column names
print(column_names)

Index(['recipe_title', 'url', 'vote_count', 'rating', 'description', 'cuisine',
       'course', 'diet', 'prep_time', 'ingredients', 'instructions', 'author',
       'tags', 'category', 'index', 'Stemmed_ingredients'],
      dtype='object')


In [18]:
food_df1.head()

,recipe_title,url,vote_count,rating,description,cuisine,course,diet,prep_time,ingredients,instructions,author,tags,category,index,Stemmed_ingredients
0,Roasted Peppers And Mushroom Tortilla Pizza Re...,https://www.archanaskitchen.com/roasted-pepper...,434,4.958525,is a quicker version pizza to satisfy your cr...,Mexican,Dinner,Vegetarian,15 M,Tortillas|Extra Virgin Olive Oil|Garlic|Mozzar...,To begin making the Roasted Peppers And Mushro...,Divya Shivaraman,Party Food Recipes|Tea Party Recipes|Mushroom ...,Pizza Recipes,1,tortilla extra virgin oliv oil garlic mozzarel...
1,Thakkali Gotsu Recipe,https://www.archanaskitchen.com/tomato-gotsu-r...,3423,4.932223,also known as the is a quick and easy to ma...,South Indian Recipes,Lunch,Vegetarian,10 M,Sesame (Gingelly) Oil|Mustard seeds (Rai/ Kadu...,To begin making Tomato Gotsu Recipe/ Thakkali ...,Archana Doshi,Vegetarian Recipes|Tomato Recipes|South Indian...,Indian Curry Recipes,2,sesam gingelli oil mustard seed rai kadugu cur...
2,Spicy Grilled Pineapple Salsa Recipe,https://www.archanaskitchen.com/spicy-grilled-...,2091,4.945959,Spicy Grilled Pineapple Salsa is a simple reci...,Mexican,Side Dish,Vegetarian,10 M,Extra Virgin Olive Oil|Pineapple|White onion|R...,To begin making the Spicy Grilled Pineapple Sa...,Archana's Kitchen,Party Starter & Appetizer Recipes|Pineapple Re...,Mexican Recipes,3,extra virgin oliv oil pineappl white onion red...
3,Toor dal Curry,https://www.archanaskitchen.com/dali-thoy-reci...,990,4.888889,The is a quintessential of Konkani dish whic...,Coastal Karnataka,Side Dish,High Protein Vegetarian,5 M,Arhar dal (Split Toor Dal)|Turmeric powder (Ha...,To prepare Karwar Style Dali Thoy Recipe (Toor...,Jyothi Rajesh,Side Dish Recipes|South Indian Recipes|Indian ...,Indian Curry Recipes,4,arhar dal split toor dal turmer powder haldi s...
4,Rajma Kofta In Milk And Poppy Seed Gravy Recipe,https://www.archanaskitchen.com/rajma-kofta-in...,345,4.828986,Koftas are traditional Indian recipes mostly w...,North Indian Recipes,Side Dish,High Protein Vegetarian,20 M,Rajma (Large Kidney Beans)|Cashew nuts|Sultana...,To begin making Rajma Kofta In Milk And Poppy ...,RUBY PATHAK,Side Dish Recipes|Indian Lunch Recipes|Office ...,Kofta Recipes,5,rajma larg kidney bean cashew nut sultana rais...


# Converting Text To Vectors

In [19]:
#Converting the Stemmed ingridients to Vectors
vocab_size = 10000
encoded_ingredients = [one_hot(d, vocab_size) for d in food_df1["Stemmed_ingredients"] ]
print(encoded_ingredients)

[[898, 4861, 2185, 1599, 1321, 3041, 8795, 5045, 3807, 5304, 2000, 1235, 6866, 7744, 1435, 7712, 1599, 8554, 6113], [4607, 1628, 1321, 4643, 7945, 6916, 4668, 6460, 4935, 3041, 3768, 1435, 3589, 1435, 5752, 9719, 1509, 8238, 9733, 6482, 1698], [4861, 2185, 1599, 1321, 4952, 2819, 1435, 3807, 5304, 2000, 1235, 6866, 7744, 2465, 8621, 4285, 8285, 4935, 4148, 4935, 8678, 6482, 6012, 6383, 7627, 8238], [7353, 6688, 4081, 6661, 6688, 1509, 8238, 9733, 6482, 9107, 3807, 4069, 4643, 7945, 6916, 4668, 3949, 8183, 6460, 4935, 4285, 8285, 4935], [1517, 9941, 9415, 1396, 5083, 3955, 6306, 7254, 3949, 8183, 8768, 7945, 8574, 3090, 5593, 1596, 3807, 4069, 8238, 4285, 8285, 4935, 9067, 7945, 9661, 6049, 474, 7926, 6482, 7858, 6641], [9246, 2584, 1321, 4837, 2610, 4355, 2188, 8910, 1737, 3210, 7945, 4306, 2978, 5172, 8596, 6269, 835, 1299, 6460, 4935, 2000, 4069, 3041, 417, 1435, 8554, 6113, 6482, 4285, 8285, 8238, 2783, 6866, 8238, 4285, 8285, 4935, 4148, 4935, 8678], [4285, 8285, 4935, 4039, 6012, 

In [20]:
#Padding is Done
max_length = 50
padded_ingredients = pad_sequences(encoded_ingredients, maxlen=max_length, padding='post')
print(padded_ingredients)

[[ 898 4861 2185 ...    0    0    0]
 [4607 1628 1321 ...    0    0    0]
 [4861 2185 1599 ...    0    0    0]
 ...
 [1509 8238 9733 ...    0    0    0]
 [ 776 6809 5391 ...    0    0    0]
 [3578 5849 3768 ...    0    0    0]]


In [21]:
 embeded_vector_size = 50

In [22]:
#Taking one Test Case
l1=["All Purpose Flour (Maida)|Ajwain (Carom seeds)|Salt|Oil|Potatoes (Aloo)|Green peas (Matar)|Roasted Peanuts (Moongphali)|Ginger|Cumin seeds (Jeera)|Fennel seeds (Saunf)|Turmeric powder (Haldi)|Red Chilli powder|Chaat Masala Powder|Amchur (Dry Mango Powder)|Black Salt (Kala Namak)|Salt|Coriander (Dhania) Leaves"]
data1 = pd.DataFrame(l1).T
data1

,0
0,All Purpose Flour (Maida)|Ajwain (Carom seeds)...


In [23]:
import pandas as pd
import numpy as np

series1=data1.iloc[0,:]
#type(series1)
series1

0    All Purpose Flour (Maida)|Ajwain (Carom seeds)...
Name: 0, dtype: object

In [24]:
single_ingredient=series1.apply(stemming)
single_ingredient

0    purpos flour maida ajwain carom seed salt oil ...
Name: 0, dtype: object

In [25]:
type(single_ingredient)

pandas.core.series.Series

In [26]:
series_single_ingredient = single_ingredient.to_string(index=False)

In [27]:
vocab_size = 10000
encoded_single_ingredient = [one_hot(series_single_ingredient, vocab_size)  ]
print(encoded_single_ingredient)

[[3487, 5593, 6420, 9625, 4398, 7945, 6482, 1321, 6280]]


In [28]:
max_length = 50
padded_single_ingredient= pad_sequences(encoded_single_ingredient, maxlen=max_length, padding='post')
print(padded_single_ingredient)

[[3487 5593 6420 9625 4398 7945 6482 1321 6280    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0]]


In [29]:
type(padded_ingredients)

numpy.ndarray

In [30]:
type(padded_single_ingredient)

numpy.ndarray

In [31]:
padded_ingredients.shape

(7963, 50)

In [32]:
padded_single_ingredient.shape

(1, 50)

# Apply Cosine_Similarity

In [33]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [34]:
# Compute the cosine similarity between the target vector and each vector in the list
similarities = cosine_similarity(padded_single_ingredient, padded_ingredients)

In [35]:
type(similarities)

numpy.ndarray

In [36]:
similarities

array([[0.43903878, 0.51459407, 0.27905647, ..., 0.58895291, 0.56428025,
        0.27875924]])

In [37]:
similarities.shape

(1, 7963)

In [38]:
# Find the index of the most similar vector
most_similar_index = np.argmax(similarities)
similarities=similarities.flatten()
similarities[most_similar_index]

0.9905479895846672

In [39]:
print(most_similar_index) #Finding Most Similar Recipie index

1060


In [40]:
print(food_df1["recipe_title"][most_similar_index]," : ",food_df1["url"][most_similar_index])

Phool Mathri Recipe  :  https://www.archanaskitchen.com/phool-mathri-recipe


In [41]:
# Find the indices of the top 10 most similar vectors
top_10_indices = np.argsort(similarities, axis=None)[:10]
similarities=similarities.flatten()
ascending_values=np.argsort(similarities)
top_10_indices=ascending_values[-1:-11:-1]
top_10_indices

array([1060, 1811, 2567,  317, 3568,  607, 7790, 7543, 5718, 5100],
      dtype=int64)

In [42]:
#print(top_ten_indices)
top_10_indices.size

10

In [43]:
for index in top_10_indices:
    print(food_df1["recipe_title"][index]," : ",food_df1["url"][index]  )
    print("-----------------------------------------------------------------")

Phool Mathri Recipe  :  https://www.archanaskitchen.com/phool-mathri-recipe
-----------------------------------------------------------------
Microwave Oats Milk Halwa Recipe  :  https://www.archanaskitchen.com/microwave-oats-milk-halwa-recipe
-----------------------------------------------------------------
Cottage Cheese And Peas Oatmeal Pancakes Recipe  :  https://www.archanaskitchen.com/cottage-cheese-and-peas-oatmeal-pancakes-recipe
-----------------------------------------------------------------
Shakkar Para Recipe   :  https://www.archanaskitchen.com/shakkarpara-shankarpali-recipe
-----------------------------------------------------------------
Til Ke Ladoo Recipe - Sesame Ladoo Recipe  :  https://www.archanaskitchen.com/til-ke-ladoo-sesame-ladoo-recipe
-----------------------------------------------------------------
Homemade Pita Bread Recipe   :  https://www.archanaskitchen.com/how-to-make-homemade-pita-bread
-----------------------------------------------------------------

# USER INPUT

In [57]:
input_ingredients=input("Please Enter Ingridients You Have :  ")
l2=[]
l2.append(input_ingredients)
data2 = pd.DataFrame(l2).T
import pandas as pd
import numpy as np

series1=data2.iloc[0,:]
type(series1)

single_ingredient=series1.apply(stemming)
series_single_ingredient = single_ingredient.to_string(index=False)
vocab_size = 10000
encoded_single_ingredient = [one_hot(series_single_ingredient, vocab_size)  ]
max_length = 50
padded_single_ingredient= pad_sequences(encoded_single_ingredient, maxlen=max_length, padding='post')
similarities = cosine_similarity(padded_single_ingredient, padded_ingredients)
similarities=similarities.flatten()
ascending_values=np.argsort(similarities)
top_10_indices=ascending_values[-1:-11:-1]
for index in top_10_indices:
    print()
    print(food_df1["recipe_title"][index]," : ",food_df1["url"][index]  )
    print("-----------------------------------------------------------------")

Please Enter Ingridients You Have :  All Purpose Flour (Maida)|Baking powder|Salt|Caster Sugar|Whole Eggs|Sunflower Oil|Vanilla Extract|Milk|Milk|Condensed Milk|Fresh cream|Double Cream|Icing Sugar|Vanilla Extract

Apple And Pomegranate Smoothie Recipe  :  https://www.archanaskitchen.com/apple-and-pomegranate-smoothie-recipe
-----------------------------------------------------------------

Kesar Elaichi Shrikand Recipe  :  https://www.archanaskitchen.com/kesar-elaichi-shrikand-recipe
-----------------------------------------------------------------

Homemade Doodh Peda Recipe   :  https://www.archanaskitchen.com/homemade-doodh-peda-milk-peda
-----------------------------------------------------------------

Custard Jelly Tart Recipe  :  https://www.archanaskitchen.com/custard-jelly-tart
-----------------------------------------------------------------

Atukula Ladoo Recipe - Sweetened Poha Energy Balls  :  https://www.archanaskitchen.com/atukula-ladoo-sweetened-poha-energy-balls-recip